In [ ]:
#!pip install -U efficientnet
# Get EfficientNet-PyTorch
#!pip install efficientnet_pytorch

#!pip install libsvm
#!pip install svm
#!pip install image-quality
#!pip download image-quality

#Installing image-quality library locally as Internet connectivity is restricted for this competition
!mkdir -p /kaggle/working/imagequality
!cp -r /kaggle/input/image-quality /kaggle/working/imagequality

!pip install /kaggle/working/imagequality/image-quality/dist/libsvm-3.23.0.4/libsvm-3.23.0.4
!pip install /kaggle/working/imagequality/image-quality/image_quality-1.2.7-py3-none-any.whl -f ./ --no-index

In [ ]:
import os

import warnings

import imquality.brisque as brisque
import PIL.Image

import pandas as pd

import glob

import multiprocessing

import json

In [ ]:
#This function is an added feature which can be used to detect and flag outlier pet images.
#Outlier pet images are images of animals categorized as illeagal to be kept as pets in Malaysia.

def get_pet_category(image_file_name):
    model = EfficientNet.from_pretrained('efficientnet-b0')
    
    # Preprocess image
    tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
    img = tfms(Image.open('img.jpg')).unsqueeze(0)
    print(img.shape) # torch.Size([1, 3, 224, 224])

    # Load ImageNet class names
    labels_map = json.load(open('../input/labelmap-final1/labels_map.txt'))
    labels_map = [labels_map[str(i)] for i in range(1000)]

    # Classify
    model.eval()
    with torch.no_grad():
        outputs = model(img)

    # Print predictions
    print('-----')
    for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
        prob = torch.softmax(outputs, dim=1)[0, idx].item()
        print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))

In [ ]:
df_test_results = pd.DataFrame()

def get_image_quality_score(full_path):
    warnings.simplefilter(action='ignore', category=FutureWarning)
    img = PIL.Image.open(full_path)
    score = round(brisque.score(img),2)
    filename = os.path.basename(full_path)
    row = {"Id":filename[:-4] , "Pawpularity":score}
    with open('/kaggle/working/submission1.csv', 'a') as convert_file:
        convert_file.write(filename[:-4])
        convert_file.write(',')
        convert_file.write(str(score))
        convert_file.write("\n")

In [ ]:
#Loop through test files and get image quality scores

'''for dirname, _, filenames in os.walk('/kaggle/input/petfinder-pawpularity-score/test'):
    for filename in filenames:
        full_path = os.path.join(dirname, filename)'''

file_list = glob.glob('/kaggle/input/petfinder-pawpularity-score/test/*.jpg')

ListOfProcesses = []
Processors = 10 # n of processors you want to use
#Divide the list of files in 'n of processors' Parts
Parts = [file_list[j:j + Processors] for j in range(0, len(file_list), Processors)]

for part in Parts:
    for f in part:
        p = multiprocessing.Process(target=get_image_quality_score, args=(str(f),))
        p.start()
        ListOfProcesses.append(p)
    for p in ListOfProcesses:
        p.join()

In [ ]:
f = open('/kaggle/working/submission1.csv','r')
newf = open('/kaggle/working/submission.csv','w')
lines = f.readlines() # read old content
new_content = "Id,Pawpularity\n"
newf.write(new_content) # write new content at the beginning
for line in lines: # write old content after new
    newf.write(line)
newf.close()
f.close()

!rm /kaggle/working/submission1.csv